In [1]:
import sys
import os
os.environ['HEASOFTPYDIR']= os.path.split(os.getcwd())[0]
import glob

import heasoftpy as hsp

from astropy.table import Table
from astropy.time import Time
from astropy.io import fits
import astropy.units as u

In [3]:
obsdir

'/Users/mcorcora/tmp/nicer/4020180460/_out'

# Set up the NICER obsid directory

We'll assume the user has downloaded the NICER observation directory for OBSID ``4142010107`` in a directory called ``/Users/mcorcora/tmp/nicer``

That is the directory ``/Users/mcorcora/tmp/nicer/4142010107`` contains the ``xti``, ``auxil`` and ``hk`` directories containing the standard archived data for the observation

We'll put the output from nicerl2 in an output directory separate from the input directory

In [2]:
nicerdatadir = os.path.join(os.environ['HOME'],'tmp/nicer')
nicerobsID = '4020180460'
obsdir = os.path.join(nicerdatadir, nicerobsID)

# place cleaned output in a separate directory
outdir =  os.path.join(os.environ['HOME'],'tmp/nicer/','nicerl2_output/'+nicerobsID+'_out')
# if outdir doesn't exist, create it
if not os.path.exists(outdir):
    os.makedirs(outdir)
    print(f'Created {outdir}')
# copy the mkf file from the input directory to the outdir
mkf = os.path.join(obsdir,'auxil',f'ni{nicerobsID}.mkf')
if os.path.exists(mkf):
    # see if mkf is gzipped    
    cmd = f'cp {mkf} {outdir}/.'
    stat=os.system(cmd)
    mkf = os.path.join(outdir, os.path.split(mkf)[1])
    print(f'Setting mkf file to {mkf}')
elif os.path.exists(mkf+'.gz'):
    #try to copy gzipped mkf     
    cmd = f'cp {mkf}.gz {outdir}/.'
    print(cmd)
    os.system(cmd)
    mkf = os.path.join(outdir, os.path.split(mkf)[1])
    print(f'Setting mkf file to {mkf}')
    #cmd = f'gunzip -f {mkf}.gz'
    #print(cmd)
    #stat=os.system(cmd)
cmd = f'chmod u+w {mkf}*'
print(cmd)
stat = os.system(cmd)


cp /Users/mcorcora/tmp/nicer/4020180460/auxil/ni4020180460.mkf.gz /Users/mcorcora/tmp/nicer/nicerl2_output/4020180460_out/.
Setting mkf file to /Users/mcorcora/tmp/nicer/nicerl2_output/4020180460_out/ni4020180460.mkf
chmod u+w /Users/mcorcora/tmp/nicer/nicerl2_output/4020180460_out/ni4020180460.mkf*


In [4]:
cwd = os.getcwd()
os.chdir(nicerdatadir)

# Create the nicerl2 task

In [7]:
tstart = Time.now()
print(f'Start at: {tstart.iso[:19]}')
nicerl2 = hsp.HSPTask('nicerl2')

nicerl2.clobber="yes"
#nicerl2.cldir=outdir
#nicerl2.mkffile=mkf
# add the KP values to the mkf file during nicerl2 processing
nicerl2.geomag_path="https://heasarc.gsfc.nasa.gov/FTP/caldb/data/gen/pcf/geomag/"
nicerl2.geomag_columns="kp_noaa.fits(KP)"

resl2 = nicerl2(indir=nicerobsID, noprompt=True, cldir=outdir, mkfile=mkf)

tend = Time.now()
print(f'End at: {tend.iso[:19]}')
print(f'nicerl2 took: {(tend.mjd-tstart.mjd)*86400} seconds')

if resl2.returncode != 0:
    print('\n')
    for o in resl2.output[:]:
        print(o)


Start at: 2021-12-18 17:30:47
End at: 2021-12-18 17:33:58
nicerl2 took: 191.64053730200976 seconds


# Analyzing NICER spectra

In [11]:
# get the on-axis rmf
res = hsp.quzcif(mission='nicer', instrument='xti',detector='-',
             filter='-', date='-', time='-',expr='-',codename='MATRIX')
rmf = [x.split()[0] for x in res.output if 'nixtiref'  in x][0]

# get the on-axis arf
res = hsp.quzcif(mission='nicer', instrument='xti',detector='-',
             filter='-', date='-', time='-',expr='-',codename='SPECRESP')
arf = [x.split()[0] for x in res.output if 'nixtiaveonaxis'  in x][0]


In [13]:
rmf, arf

('https://heasarc.gsfc.nasa.gov/FTP/caldb/data/nicer/xti/cpf/rmf/nixtiref20170601v003.rmf',
 'https://heasarc.gsfc.nasa.gov/FTP/caldb/data/nicer/xti/cpf/arf/nixtiaveonaxis20170601v005.arf')